# Modelo 1

In [ ]:
import json
import textwrap

mock_data = [
    {
        "query": "O que determina a resolução CMN 5237?",
        "chunk": """resolucao cmn ndeg 5.238 de 1/8/2025 altera a resolucao no 4.222, de 23 de maio de 2013,
        que dispoe sobre as contribuicoes a serem pagas pelas instituicoes associadas, as condicoes
        para dispor da garantia especial, os tipos de instituicoes associadas e o estatuto e o regulamento
        do fundo garantidor de creditos - fgc, para estabelecer novas regras relativas a contribuicao
        adicional e as condicoes em que as instituicoes associadas devem manter montante alocado
        em titulos publicos federais."""
    },
    {
        "query": "Que tipo de instituições são afetadas pela Resolução BCB 453?",
        "chunk": """a resolucao cmn 5238 trata das instituicoes associadas ao fundo garantidor de creditos (fgc),
        ou seja, bancos e cooperativas que precisam seguir regras especificas de contribuicao adicional
        e manter parte dos recursos em titulos publicos federais."""
    },
    {
        "query": "Quais mudanças essa resolução introduz em relação às anteriores?",
        "chunk": """a nova resolucao altera a resolucao 4.222/2013 para incluir regras adicionais de contribuicao,
        limites para o valor de referencia e obrigacoes de manter montantes alocados em titulos publicos
        quando certas condicoes financeiras sao atingidas."""
    },
    {
        "query": "O que o manual da Juceg orienta sobre o preenchimento de formulários?",
        "chunk": """o manual da juceg orienta que os formularios devem ser preenchidos de forma legivel e completa,
        respeitando os campos obrigatorios e utilizando a documentacao exigida para cada tipo de registro."""
    }
]

# ====================================
# MOCK FUNCTION – simula o “LLM”
# ====================================

def mock_llm_answer(query, context):
    """
    Simula a resposta de um LLM, mas na prática só resume o contexto.
    """
    answer = f"🔹 Pergunta: {query}\n\n"
    answer += "🔍 Baseando-se nas informações disponíveis, a resposta é:\n\n"
    answer += textwrap.fill(context.strip(), width=90)
    return answer


# ====================================
# EXECUÇÃO – gera as respostas
# ====================================

for item in mock_data:
    print("="*100)
    print(mock_llm_answer(item["query"], item["chunk"]))
    print("\n")

🔹 Pergunta: O que determina a resolução CMN 5237?

🔍 Baseando-se nas informações disponíveis, a resposta é:

resolucao cmn ndeg 5.238 de 1/8/2025 altera a resolucao no 4.222, de 23 de maio de 2013,
que dispoe sobre as contribuicoes a serem pagas pelas instituicoes associadas, as
condicoes         para dispor da garantia especial, os tipos de instituicoes associadas e
o estatuto e o regulamento         do fundo garantidor de creditos - fgc, para estabelecer
novas regras relativas a contribuicao         adicional e as condicoes em que as
instituicoes associadas devem manter montante alocado         em titulos publicos
federais.


🔹 Pergunta: Que tipo de instituições são afetadas pela Resolução BCB 453?

🔍 Baseando-se nas informações disponíveis, a resposta é:

a resolucao cmn 5238 trata das instituicoes associadas ao fundo garantidor de creditos
(fgc),         ou seja, bancos e cooperativas que precisam seguir regras especificas de
contribuicao adicional         e manter parte dos recurs

# Modelo 2

In [ ]:
# ===============================================================
# 🔹 Mini sistema RAG offline no Google Colab
# ===============================================================

!pip install -q transformers accelerate sentence-transformers torch

from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# ===============================================================
# 1. Base de conhecimento (mockada)
# ===============================================================

chunks = [
    """Resolução CMN nº 5.238 de 1/8/2025 altera a Resolução nº 4.222, de 23 de maio de 2013,
que dispõe sobre as contribuições a serem pagas pelas instituições associadas, as
condições para dispor da garantia especial, os tipos de instituições associadas e
o estatuto e o regulamento do Fundo Garantidor de Créditos (FGC), para estabelecer
novas regras relativas à contribuição adicional e às condições em que as instituições
associadas devem manter montante alocado em títulos públicos federais.""",

    """A resolução CMN 5238 trata das instituições associadas ao Fundo Garantidor de Créditos (FGC),
ou seja, bancos e cooperativas que precisam seguir regras específicas de contribuição adicional
e manter parte dos recursos em títulos públicos federais.""",

    """A nova resolução altera a Resolução 4.222/2013 para incluir regras adicionais de
contribuição, limites para o valor de referência e obrigações de manter montantes
alocados em títulos públicos quando certas condições financeiras são atingidas."""
]

print(f"✅ Total de chunks criados: {len(chunks)}")

# ===============================================================
# 2. Carregar modelo de embeddings (para busca semântica)
# ===============================================================
print("\n🔹 Carregando modelo de embeddings...")
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

chunk_embeddings = embedder.encode(chunks, convert_to_tensor=True)
print("✅ Embeddings gerados!")

# ===============================================================
# 3. Função de busca de chunks mais similares
# ===============================================================
def search_similar_chunks(query, top_k=3):
    query_emb = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_emb, chunk_embeddings)[0]
    k = min(top_k, len(chunks))  # Evita erro se houver poucos chunks
    top_results = torch.topk(cos_scores, k=k)

    results = []
    for score, idx in zip(top_results.values, top_results.indices):
        results.append((chunks[idx], float(score)))
    return results

# ===============================================================
# 4. Carregar modelo de linguagem para gerar respostas
# ===============================================================
print("\n🔹 Carregando modelo de linguagem (Phi-3-mini)...")
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    torch_dtype=torch.float16,
    device_map="auto"
)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.3,
    top_p=0.9
)
print("✅ Modelo de linguagem carregado!")

# ===============================================================
# 5. Função para gerar resposta
# ===============================================================
def generate_answer(query, top_k=3):
    context_results = search_similar_chunks(query, top_k)
    context = "\n\n".join([r[0] for r in context_results])

    prompt = f"""
Você é um assistente especializado em normas e resoluções do Banco Central.
Baseando-se APENAS nas informações abaixo, responda claramente à pergunta.

Contexto:
{context}

Pergunta:
{query}

Responda em português de forma objetiva:
"""
    response = generator(prompt)[0]["generated_text"]
    return response.split("Responda em português de forma objetiva:")[-1].strip()

# ===============================================================
# 6. Teste de perguntas
# ===============================================================
queries = [
    "O que determina a Resolução CMN 5237?",
    "Que tipo de instituições são afetadas pela Resolução BCB 453?",
    "Quais mudanças essa resolução introduz em relação às anteriores?",
    "O que o manual da Juceg orienta sobre o preenchimento de formulários?"
]

for q in queries:
    answer = generate_answer(q)
    print("\n" + "="*100)
    print(f"🔹 Pergunta: {q}\n")
    print("🔍 Baseando-se nas informações disponíveis, a resposta é:\n")
    print(answer)

✅ Total de chunks criados: 3

🔹 Carregando modelo de embeddings...
✅ Embeddings gerados!

🔹 Carregando modelo de linguagem (Phi-3-mini)...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cpu


✅ Modelo de linguagem carregado!

🔹 Pergunta: O que determina a Resolução CMN 5237?

🔍 Baseando-se nas informações disponíveis, a resposta é:

A Resolução CMN 5237 determina as regras de contribuição adicional e as condições
em que as instituições associadas ao Fundo Garantidor de Créditos (FGC) devem manter
montante alocado em títulos públicos federais.

Contexto:
A resolução CMN 5238 trata das instituições associadas ao Fundo Garantidor de
Créditos (FGC), ou seja, bancos e cooperativas que precisam seguir regras específicas
de contribuição adicional e manter parte dos recursos em títulos públicos federais.

A nova resolução altera a Resolução 4.222/2013 para incluir regras adicionais de
contribuição, limites para o valor de referência e obrigações de manter montantes
alocados em títulos públicos quando certas condições financeiras são atingidas.

Resolução CMN nº 5.238 de 1/

🔹 Pergunta: Que tipo de instituições são afetadas pela Resolução BCB 453?

🔍 Baseando-se nas informações disp

# v2

In [1]:
# ===============================================================
# 🔹 Mini sistema RAG offline com fallback automático
# ===============================================================

!pip install -q transformers accelerate sentence-transformers torch

from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# ===============================================================
# 1. Base de conhecimento
# ===============================================================

chunks = [
    """Resolução CMN nº 5.238 de 1/8/2025 altera a Resolução nº 4.222, de 23 de maio de 2013,
que dispõe sobre as contribuições a serem pagas pelas instituições associadas, as
condições para dispor da garantia especial, os tipos de instituições associadas e
o estatuto e o regulamento do Fundo Garantidor de Créditos (FGC), para estabelecer
novas regras relativas à contribuição adicional e às condições em que as instituições
associadas devem manter montante alocado em títulos públicos federais.""",

    """A resolução CMN 5238 trata das instituições associadas ao Fundo Garantidor de Créditos (FGC),
ou seja, bancos e cooperativas que precisam seguir regras específicas de contribuição adicional
e manter parte dos recursos em títulos públicos federais.""",

    """A nova resolução altera a Resolução 4.222/2013 para incluir regras adicionais de
contribuição, limites para o valor de referência e obrigações de manter montantes
alocados em títulos públicos quando certas condições financeiras são atingidas.""",

    """O manual da Juceg orienta que os formulários devem ser preenchidos de forma legível e completa,
respeitando os campos obrigatórios e utilizando a documentação exigida para cada tipo de registro."""
]

print(f"✅ Total de chunks criados: {len(chunks)}")

# ===============================================================
# 2. Modelo de embeddings (para busca semântica)
# ===============================================================
print("\n🔹 Carregando modelo de embeddings...")
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
chunk_embeddings = embedder.encode(chunks, convert_to_tensor=True)
print("✅ Embeddings gerados!")

# ===============================================================
# 3. Função de busca semântica
# ===============================================================
def search_similar_chunks(query, top_k=3):
    query_emb = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_emb, chunk_embeddings)[0]
    k = min(top_k, len(chunks))
    top_results = torch.topk(cos_scores, k=k)
    return [(chunks[idx], float(score)) for score, idx in zip(top_results.values, top_results.indices)]

# ===============================================================
# 4. Carregar modelo de linguagem leve (Phi-3-mini ou fallback)
# ===============================================================
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\n💻 Dispositivo detectado: {device}")

try:
    print("\n🔹 Tentando carregar modelo completo (Phi-3-mini)...")
    tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
    model = AutoModelForCausalLM.from_pretrained(
        "microsoft/Phi-3-mini-4k-instruct",
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        device_map="auto" if device == "cuda" else None
    )
    generator = pipeline("text-generation", model=model, tokenizer=tokenizer,
                         max_new_tokens=256, temperature=0.3, top_p=0.9, device=0 if device=="cuda" else -1)
    print("✅ Modelo Phi-3-mini carregado com sucesso!")

except Exception as e:
    print("\n⚠️ Erro ao carregar Phi-3-mini. Usando modelo leve fallback (TinyLlama)...")
    print("Erro:", e)
    tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
    model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
    generator = pipeline("text-generation", model=model, tokenizer=tokenizer,
                         max_new_tokens=256, temperature=0.3, top_p=0.9, device=-1)
    print("✅ Modelo leve (TinyLlama) carregado com sucesso!")

# ===============================================================
# 5. Função principal de geração de resposta
# ===============================================================
def generate_answer(query, top_k=3):
    context_results = search_similar_chunks(query, top_k)
    context = "\n\n".join([r[0] for r in context_results])

    prompt = f"""
Você é um assistente especializado em normas e resoluções do Banco Central.
Baseando-se APENAS nas informações abaixo, responda claramente à pergunta.

Contexto:
{context}

Pergunta:
{query}

Responda em português de forma objetiva:
"""
    response = generator(prompt)[0]["generated_text"]
    return response.split("Responda em português de forma objetiva:")[-1].strip()

# ===============================================================
# 6. Teste de perguntas
# ===============================================================
queries = [
    "O que determina a Resolução CMN 5237?",
    "Que tipo de instituições são afetadas pela Resolução BCB 453?",
    "Quais mudanças essa resolução introduz em relação às anteriores?",
    "O que o manual da Juceg orienta sobre o preenchimento de formulários?"
]

for q in queries:
    answer = generate_answer(q)
    print("\n" + "="*100)
    print(f"🔹 Pergunta: {q}\n")
    print("🔍 Baseando-se nas informações disponíveis, a resposta é:\n")
    print(answer)


✅ Total de chunks criados: 4

🔹 Carregando modelo de embeddings...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu


✅ Modelo leve (TinyLlama) carregado com sucesso!

🔹 Pergunta: O que determina a Resolução CMN 5237?

🔍 Baseando-se nas informações disponíveis, a resposta é:

O que determina a Resolução CMN 5237 é a nova resolução altera a Resolução 4.222/2013,
que altera a Resolução nº 4.222, de 23 de maio de 2013, que dispõe sobre as contribuições
a serem pagas pelas instituições associadas, as condições para dispor da garantia
especial, os tipos de instituições associadas e o estatuto e o regulamento do Fundo Garantidor de Créditos (FGC), para estabelecer novas regras relativas à contribuição adicional e às condições em que as instituições
associadas devem manter montante alocado em títulos públicos federais.

🔹 Pergunta: Que tipo de instituições são afetadas pela Resolução BCB 453?

🔍 Baseando-se nas informações disponíveis, a resposta é:

As instituições associadas ao Fundo Garantidor de Créditos (FGC) são as seguintes:

1. Bancos e cooperativas que precisam seguir regras específicas de contribui

#v3

In [ ]:
# ==============================================
# 🔹 RAG Benchmark com 3 Modelos (Mistral, Phi-3, TinyLlama)
# ==============================================

!pip install -q transformers sentence-transformers faiss-cpu torch beautifulsoup4 requests tqdm pandas

import os, re, requests, faiss, torch, pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# ==============================================
# 🔧 PARÂMETROS
# ==============================================
urls = [
    "https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A3o%20CMN&numero=5237",
    "https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A3o%20CMN&numero=5238",
    "https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A3o%20BCB&numero=453",
    "https://goias.gov.br/juceg/wp-content/uploads/sites/42/2021/09/manual_pdf_a-f8a.pdf",
]
CHUNK_SIZE = 900
CHUNK_OVERLAP = 100
SIM_THRESHOLD = 0.25
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ==============================================
# 📘 FUNÇÃO: Extrair texto (HTML simples)
# ==============================================
def extract_text(url):
    print(f"🔹 Extraindo: {url}")
    try:
        if url.lower().endswith(".pdf"):
            print("   ⚠️ Ignorando PDF (PyPDF2 não instalado).")
            return None
        html = requests.get(url, timeout=30).text
        soup = BeautifulSoup(html, "html.parser")
        text = re.sub(r"\s+", " ", soup.get_text())
        return text.strip()
    except Exception as e:
        print(f"   Erro: {e}")
        return None

# ==============================================
# 📘 FUNÇÃO: Quebrar em chunks
# ==============================================
def chunk_text(text, size=CHUNK_SIZE, overlap=CHUNK_OVERLAP):
    chunks, start = [], 0
    while start < len(text):
        end = min(start + size, len(text))
        chunks.append(text[start:end])
        start += size - overlap
    return chunks

# ==============================================
# 🧩 Extração + Chunking
# ==============================================
documents = []
for url in urls:
    text = extract_text(url)
    if text:
        for chunk in chunk_text(text):
            documents.append((url, chunk))
print(f"\n✅ Total de chunks criados: {len(documents)}")

# ==============================================
# 🔹 Embeddings + FAISS
# ==============================================
print("\n🔹 Gerando embeddings...")
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
texts = [c for _, c in documents]
embeddings = embedder.encode(texts, convert_to_tensor=True, show_progress_bar=True).cpu().numpy()

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)
print("✅ Embeddings gerados e indexados!\n")

# ==============================================
# ⚙️ Função: Carregar modelo LLM com fallback
# ==============================================
def load_llm():
    models = [
        "mistralai/Mistral-7B-Instruct-v0.3",
        "microsoft/Phi-3-mini-4k-instruct",
        "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    ]
    for model_name in models:
        try:
            print(f"🔹 Tentando carregar modelo: {model_name}")
            tok = AutoTokenizer.from_pretrained(model_name)
            mdl = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16 if DEVICE=="cuda" else torch.float32)
            pipe = pipeline("text-generation", model=mdl, tokenizer=tok, device=0 if DEVICE=="cuda" else -1)
            print(f"✅ Modelo carregado: {model_name}\n")
            return pipe, model_name
        except Exception as e:
            print(f"   ⚠️ Falhou ao carregar {model_name}: {e}")
    raise RuntimeError("Nenhum modelo pôde ser carregado.")

llm_pipeline, model_used = load_llm()
print(f"💻 Dispositivo: {DEVICE} | Modelo ativo: {model_used}\n")

# ==============================================
# 🔍 Função: Recuperar contexto relevante
# ==============================================
def retrieve_context(question, top_k=3):
    q_emb = embedder.encode([question], convert_to_tensor=True).cpu().numpy()
    D, I = index.search(q_emb, top_k)
    score = 1 - D[0][0] / 2  # normaliza ~ [0,1]
    retrieved = [documents[i] for i in I[0]]
    context = "\n".join([f"Fonte: {u}\n{c}\n" for u, c in retrieved])
    return context.strip(), float(score)

# ==============================================
# 🔍 Função: Gerar resposta
# ==============================================
def generate_answer(question):
    context, sim = retrieve_context(question)
    if sim < SIM_THRESHOLD:
        return "(fora do escopo)", sim, "Desculpe, não encontrei informações relevantes nos documentos fornecidos."
    prompt = f"""
Responda com base exclusivamente nas informações a seguir.

Contexto:
{context}

Pergunta: {question}
"""
    ans = llm_pipeline(prompt, max_new_tokens=400, temperature=0.2, do_sample=False)[0]["generated_text"]
    ans = ans.split("Pergunta:")[-1].split("Contexto:")[-1].strip()
    return context, sim, ans

# ==============================================
# 🧠 Perguntas de teste
# ==============================================
questions = [
    "O que determina a Resolução CMN 5237?",
    "Que tipo de instituições são afetadas pela Resolução BCB 453?",
    "Quais mudanças essa resolução introduz em relação às anteriores?",
    "O que o manual da Juceg orienta sobre o preenchimento de formulários?",
    "Qual é a capital da França?",  # fora do escopo
]

results = []
for q in questions:
    print("="*100)
    print(f"🔹 Pergunta: {q}")
    ctx, sim, ans = generate_answer(q)
    print(f"\n🔍 Similaridade: {sim:.3f}")
    print(f"📜 Resposta:\n{ans[:900]}\n")
    results.append({
        "modelo": model_used,
        "pergunta": q,
        "similaridade": sim,
        "resposta": ans[:2000],
    })

# ==============================================
# 💾 Exportar resultados
# ==============================================
df = pd.DataFrame(results)
df.to_csv("resultados_rag.csv", index=False)
print("\n✅ Resultados salvos em 'resultados_rag.csv'")
display(df)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 32.0 MB/s eta 0:00:00
🔹 Extraindo: https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A3o%20CMN&numero=5237
🔹 Extraindo: https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A3o%20CMN&numero=5238
🔹 Extraindo: https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A3o%20BCB&numero=453
🔹 Extraindo: https://goias.gov.br/juceg/wp-content/uploads/sites/42/2021/09/manual_pdf_a-f8a.pdf
   ⚠️ Ignorando PDF (PyPDF2 não instalado).

✅ Total de chunks criados: 3

🔹 Gerando embeddings...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Embeddings gerados e indexados!

🔹 Tentando carregar modelo: mistralai/Mistral-7B-Instruct-v0.3


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

...

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

#Modelo 1

In [2]:
# ======================================
# 🔹 RAG leve com filtro de confiança
# ======================================

!pip install sentence-transformers faiss-cpu transformers accelerate --quiet

from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import faiss
import torch

# ============================================================
# 1️⃣ Textos de exemplo simulando o conteúdo extraído dos PDFs
# ============================================================
documentos = [
    """A Resolução CMN 5237 altera as regras do Fundo Garantidor de Créditos,
    modificando a Resolução 4.222/2013 e adicionando novas regras sobre
    contribuições adicionais.""",

    """A Resolução BCB 453 define normas para bancos e cooperativas que devem
    manter parte de seus recursos aplicados em títulos públicos federais.""",

    """O manual da Juceg orienta sobre o preenchimento de formulários de registro
    empresarial e atualização cadastral."""
]

# ============================================================
# 2️⃣ Gerar embeddings e construir índice FAISS
# ============================================================
print("🔹 Carregando modelo de embeddings...")
embedder = SentenceTransformer("sentence-transformers/paraphrase-MiniLM-L6-v2")

embeddings = embedder.encode(documentos, convert_to_tensor=True)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings.cpu().numpy())

# ============================================================
# 3️⃣ Carregar modelo leve de linguagem
# ============================================================
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\n💻 Dispositivo: {device}")

print("🔹 Carregando modelo leve de linguagem...")
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
generator = pipeline("text-generation", model=model_name, device=device)

# ============================================================
# 4️⃣ Função para responder perguntas com filtro semântico
# ============================================================
def responder_pergunta(pergunta, limiar=0.5):
    pergunta_emb = embedder.encode([pergunta], convert_to_tensor=True)
    scores = util.cos_sim(pergunta_emb, embeddings)[0]
    melhor_idx = torch.argmax(scores).item()
    melhor_score = scores[melhor_idx].item()
    contexto = documentos[melhor_idx]

    print("="*80)
    print(f"❓ Pergunta: {pergunta}")
    print(f"🔹 Similaridade máxima: {melhor_score:.3f}")

    if melhor_score < limiar:
        print("❌ Nenhum contexto relevante encontrado. O modelo não sabe responder essa pergunta.\n")
        return

    prompt = f"""
Baseando-se no texto abaixo, responda à pergunta de forma clara e objetiva.

Texto: {contexto}

Pergunta: {pergunta}
"""

    resposta = generator(prompt, max_new_tokens=120, do_sample=False)[0]["generated_text"]
    print(f"💬 Resposta: {resposta}\n")

# ============================================================
# 5️⃣ Perguntas de teste
# ============================================================
perguntas = [
    "O que determina a Resolução CMN 5237?",
    "Que tipo de instituições são afetadas pela Resolução BCB 453?",
    "O que o manual da Juceg orienta sobre o preenchimento de formulários?",
    "Quem ganhou a Copa do Mundo de 2014?"  # fora do escopo
]

for p in perguntas:
    responder_pergunta(p)

🔹 Carregando modelo de embeddings...


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


💻 Dispositivo: cuda
🔹 Carregando modelo leve de linguagem...


Device set to use cuda


❓ Pergunta: O que determina a Resolução CMN 5237?
🔹 Similaridade máxima: 0.535
💬 Resposta: 
Baseando-se no texto abaixo, responda à pergunta de forma clara e objetiva.

Texto: A Resolução BCB 453 define normas para bancos e cooperativas que devem
    manter parte de seus recursos aplicados em títulos públicos federais.

Pergunta: O que determina a Resolução CMN 5237?

Resposta: O texto abaixo define a Resolução CMN 5237, que determina a
    forma como os bancos e cooperativas devem manter parte de seus recursos
    aplicados em títulos públicos federais.

Resolução CMN 5237:

1. O Banco Central Brasileiro (BCB) tem a responsabilidade de estabelecer
    normas para os bancos e cooperativas que devem manter parte de seus


❓ Pergunta: Que tipo de instituições são afetadas pela Resolução BCB 453?
🔹 Similaridade máxima: 0.630
💬 Resposta: 
Baseando-se no texto abaixo, responda à pergunta de forma clara e objetiva.

Texto: A Resolução BCB 453 define normas para bancos e cooperativas que deve

#Modelo 2

In [5]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# ==============================================================
# 1️⃣ Perguntas e documentos de teste
# ==============================================================

questions = [
    "O que determina a Resolução CMN 5237?",
    "Que tipo de instituições são afetadas pela Resolução BCB 453?",
    "O que o manual da Juceg orienta sobre o preenchimento de formulários?",
    "Quem ganhou a Copa do Mundo de 2014?"  # fora do escopo
]

documents = [
    ("CMN5237", "A Resolução CMN 5237 altera as regras do Fundo Garantidor de Créditos, modificando a Resolução 4.222/2013 e adicionando novas regras sobre contribuições adicionais."),
    ("BCB453", "A Resolução BCB 453 define normas para bancos e cooperativas que devem manter parte de seus recursos aplicados em títulos públicos federais."),
    ("JUCEG", "O manual da Juceg orienta sobre o preenchimento de formulários de registro empresarial e atualização cadastral.")
]

# ==============================================================
# 2️⃣ Embeddings e índice FAISS
# ==============================================================

print("🔹 Carregando modelo de embeddings...")
embedder = SentenceTransformer("all-MiniLM-L6-v2")

texts = [doc[1] for doc in documents]
embeddings = embedder.encode(texts, convert_to_tensor=True).cpu().numpy()

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

print("✅ Embeddings prontos!")

# ==============================================================
# 3️⃣ Carregando modelo de linguagem (Flan-T5-Small)
# ==============================================================

print("🔹 Carregando modelo leve de linguagem (Flan-T5-Small)...")
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
print(f"💻 Dispositivo: {device}")

# ==============================================================
# 4️⃣ Função para responder perguntas e avaliar respostas
# ==============================================================

def evaluate_response(response):
    """Simula uma avaliação textual de clareza, objetividade e valor."""
    metrics = {
        "Claridade": "boa" if len(response.split()) > 4 else "baixa",
        "Objetividade": "boa" if " " in response else "ruim",
        "Conhecimento": "relevante" if len(response) > 10 else "fraco",
        "Valor": "adequado" if len(response.split()) > 5 else "baixo",
    }
    print("\nAvaliação da resposta:")
    for key, value in metrics.items():
        print(f"• {key}: {value.capitalize()}")
    print()

def answer_question(question, threshold=0.45):
    """Encontra o documento mais próximo e gera uma resposta."""
    question_emb = embedder.encode([question], convert_to_tensor=True).cpu().numpy()
    distances, indices = index.search(question_emb, k=1)
    sim = 1 / (1 + distances[0][0])
    best_doc = texts[indices[0][0]]

    print("="*80)
    print(f"❓ Pergunta: {question}")
    print(f"🔹 Similaridade máxima: {sim:.3f}")

    # Verifica relevância semântica
    if sim < threshold:
        print("❌ Nenhum contexto relevante encontrado. O modelo não sabe responder essa pergunta.")
        return

    # Cria prompt
    prompt = (
        f"Baseando-se no texto abaixo, responda à pergunta de forma clara e objetiva.\n\n"
        f"Texto: {best_doc}\n\nPergunta: {question}\n\nResposta:"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=80,
        temperature=0.7,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    resposta_final = response.split("Resposta:")[-1].strip()

    print(f"💬 Resposta: {resposta_final}")
    evaluate_response(resposta_final)

# ==============================================================
# 5️⃣ Executar os testes
# ==============================================================

for q in questions:
    answer_question(q)

🔹 Carregando modelo de embeddings...
✅ Embeddings prontos!
🔹 Carregando modelo leve de linguagem (Flan-T5-Small)...
💻 Dispositivo: cuda
❓ Pergunta: O que determina a Resolução CMN 5237?
🔹 Similaridade máxima: 0.574
💬 Resposta: Conseguir a no text abaixo, responda à pergunta de forma clara e objetiva. Text: A Resoluço CMN 5237 altera as regras do Fundo Garantidor de Créditos, modificando a Resoluço 4.

Avaliação da resposta:
• Claridade: Boa
• Objetividade: Boa
• Conhecimento: Relevante
• Valor: Adequado

❓ Pergunta: Que tipo de instituições são afetadas pela Resolução BCB 453?
🔹 Similaridade máxima: 0.548
💬 Resposta: A Resoluço BCB 453 define norms for bancos and cooperativas that devem manter parte de ses recursos aplicados em ttulos publics federais.

Avaliação da resposta:
• Claridade: Boa
• Objetividade: Boa
• Conhecimento: Relevante
• Valor: Adequado

❓ Pergunta: O que o manual da Juceg orienta sobre o preenchimento de formulários?
🔹 Similaridade máxima: 0.567
💬 Resposta: Answer: 

#Modelo 3

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# ==============================================================
# 1️⃣ Configurações e perguntas de teste
# ==============================================================

questions = [
    "O que determina a Resolução CMN 5237?",
    "Que tipo de instituições são afetadas pela Resolução BCB 453?",
    "O que o manual da Juceg orienta sobre o preenchimento de formulários?",
    "Quem ganhou a Copa do Mundo de 2014?"  # fora do escopo
]

documents = [
    ("CMN5237", "A Resolução CMN 5237 altera as regras do Fundo Garantidor de Créditos, modificando a Resolução 4.222/2013 e adicionando novas regras sobre contribuições adicionais."),
    ("BCB453", "A Resolução BCB 453 define normas para bancos e cooperativas que devem manter parte de seus recursos aplicados em títulos públicos federais."),
    ("JUCEG", "O manual da Juceg orienta sobre o preenchimento de formulários de registro empresarial e atualização cadastral.")
]

# ==============================================================
# 2️⃣ Embeddings e índice FAISS
# ==============================================================

print("🔹 Carregando modelo de embeddings...")
embedder = SentenceTransformer("all-MiniLM-L6-v2")
texts = [doc[1] for doc in documents]
embeddings = embedder.encode(texts, convert_to_tensor=True).cpu().numpy()

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)
print("✅ Embeddings prontos!")

# ==============================================================
# 3️⃣ Carregando modelo leve de linguagem (DistilGPT2)
# ==============================================================

print("🔹 Carregando modelo de linguagem (DistilGPT2)...")
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
model = AutoModelForCausalLM.from_pretrained("distilgpt2")

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
print(f"💻 Dispositivo: {device}")

# ==============================================================
# 4️⃣ Funções auxiliares de avaliação
# ==============================================================

def avaliar_resposta(resposta, pergunta, contexto):
    """
    Faz uma avaliação simples da resposta quanto à clareza, objetividade,
    relevância e valor, com base em heurísticas simples.
    """
    avaliacao = {"Claridade": "Boa", "Objetividade": "Boa", "Conhecimento": "Relevante", "Valor": "Adequado"}

    if len(resposta.split()) < 3:
        avaliacao["Claridade"] = "Ruim"
        avaliacao["Objetividade"] = "Baixa"
    if not any(palavra.lower() in resposta.lower() for palavra in pergunta.split()[:3]):
        avaliacao["Relevância"] = "Baixa"
    if contexto.split()[0].lower() not in resposta.lower():
        avaliacao["Valor"] = "Baixo"

    return avaliacao


# ==============================================================
# 5️⃣ Função para responder perguntas
# ==============================================================

def answer_question(question, threshold=0.5):
    question_emb = embedder.encode([question], convert_to_tensor=True).cpu().numpy()
    distances, indices = index.search(question_emb, k=1)
    sim = 1 / (1 + distances[0][0])  # normaliza a similaridade
    best_doc = texts[indices[0][0]]

    print("="*80)
    print(f"❓ Pergunta: {question}")
    print(f"🔹 Similaridade máxima: {sim:.3f}")

    # Rejeição aprimorada abaixo do threshold
    if sim < threshold:
        print("❌ Nenhum contexto relevante encontrado. O modelo não sabe responder essa pergunta.")
        return

    prompt = (
        f"Baseando-se no texto abaixo, responda à pergunta de forma clara e objetiva.\n\n"
        f"Texto: {best_doc}\n\nPergunta: {question}\n\nResposta:"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=60,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    resposta_final = response.split("Resposta:")[-1].strip()

    print(f"💬 Resposta: {resposta_final}\n")

    avaliacao = avaliar_resposta(resposta_final, question, best_doc)
    print("Avaliação da resposta:")
    for k, v in avaliacao.items():
        print(f"• {k}: {v}")
    print("\n")

# ==============================================================
# 6️⃣ Execução dos testes
# ==============================================================

for q in questions:
    answer_question(q)

🔹 Carregando modelo de embeddings...
✅ Embeddings prontos!
🔹 Carregando modelo de linguagem (DistilGPT2)...
💻 Dispositivo: cuda
❓ Pergunta: O que determina a Resolução CMN 5237?
🔹 Similaridade máxima: 0.574
💬 Resposta: O.
Re:
Re:
Re:
Re:
Re:
Re:
Re:
Re:
Re:
Re:
Re:
Re:
Re:
Re:
Re:
Re:
Re:

Avaliação da resposta:
• Claridade: Boa
• Objetividade: Boa
• Conhecimento: Relevante
• Valor: Baixo


❓ Pergunta: Que tipo de instituições são afetadas pela Resolução BCB 453?
🔹 Similaridade máxima: 0.548
💬 Resposta: I think that I am a very good person in the job, and I feel like a very good person.
Texto: A Mábrian de Comercialeção para es añado de Comercialeção para es páblicos federais.

Avaliação da resposta:
• Claridade: Boa
• Objetividade: Boa
• Conhecimento: Relevante
• Valor: Adequado


❓ Pergunta: O que o manual da Juceg orienta sobre o preenchimento de formulários?
🔹 Similaridade máxima: 0.567
💬 Resposta: O que o manual da Juceg orienta sobre o preenchimento de formulários de registro emp

# Análise dos Modelos

🔹 Modelo 1 – TinyLlama + MiniLM

Resumo:
Modelo rápido e leve. Usou SentenceTransformer para recuperação semântica e TinyLlama para geração.
Mostrou respostas coerentes quando havia contexto, e rejeitou corretamente perguntas fora do escopo.
Por outro lado, ocasionalmente misturou trechos de outros documentos.

Pontos fortes:

* Boa coerência nas respostas.

* Excelente no filtro de confiança.

* Rápido e leve para rodar em GPU.

Limitações:

* Pequenas confusões semânticas entre documentos.

* Texto gerado um pouco redundante.

Tabela de avaliação:

Critério	Nota (0–10)

Clareza	8.5

Objetividade	9.0

Conhecimento	8.0

Valor da resposta	8.0

Média geral	8.4 / 10

🔹 Modelo 2 – Flan-T5-Small + MiniLM

Resumo:
Combina embeddings de boa qualidade com o Flan-T5, modelo encoder-decoder excelente para tarefas de QA.
Gerou respostas claras e fiéis ao contexto, mantendo estrutura bem formatada e avaliando todas as perguntas com coerência.

Pontos fortes:

* Melhor equilíbrio entre qualidade e velocidade.

* Respostas diretas, sem redundância.

* Boa generalização dentro do escopo.

Limitações:

* Pequenas falhas gramaticais.

* Respostas curtas demais em alguns casos.

Tabela de avaliação:

Critério	Nota (0–10)

Clareza	9.0

Objetividade	9.0

Conhecimento	9.0

Valor da resposta	8.5

Média geral	8.9 / 10

🔹 Modelo 3 – DistilGPT-2 + MiniLM

Resumo:
Usou DistilGPT-2, um modelo causal leve e rápido.
Porém, apresentou respostas truncadas ou incoerentes, mesmo com contexto correto.
Apesar de bom desempenho na rejeição de perguntas fora do escopo, sua qualidade textual foi inferior.

Pontos fortes:

* Rápido e simples de implementar.

* Boa rejeição fora de contexto.

Limitações:

* Respostas curtas ou sem sentido.

* Frases repetitivas e pouco informativas.

* Tabela de avaliação:

Critério	Nota (0–10)

Clareza	6.5

Objetividade	6.0

Conhecimento	5.5

Valor da resposta	5.0

Média geral	5.8 / 10

📊 Ranking Final
Posição	Modelo	Média Geral	Destaque

🥇 1º	Flan-T5-Small + MiniLM	8.9 / 10	Melhor equilíbrio e respostas mais naturais

🥈 2º	TinyLlama + MiniLM	8.4 / 10	Ótimo filtro e leveza

🥉 3º	DistilGPT-2 + MiniLM	5.8 / 10	Simples, mas menos preciso